In [1]:
# Reference: 
# 1. https://colab.research.google.com/drive/1wm8Z0ui8ZGSXoR50x52GvWMQkfSPPJ-T#scrollTo=SCFXYdbIXguc
# 2. https://colab.research.google.com/drive/1YRHK4HO8RktGzlYmGjBo056kzVD4_j9o#scrollTo=BJR6t_gCQe_x

!pip install datasets
!pip install transformers

import torch
from pprint import pprint
from datasets import load_dataset
import numpy
import pandas as pd
import io
import random
import numpy as np

# Change here if use own drive
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/MyDrive

# Set the seed value all over the place to make this reproducible.
def setup_seed(seed):
    random.seed(seed)                          
    np.random.seed(seed)                       
    torch.manual_seed(seed)                    
    torch.cuda.manual_seed(seed)               
    torch.cuda.manual_seed_all(seed)           
    torch.backends.cudnn.deterministic = True  
    torch.backends.cudnn.benchmark = False

setup_seed(42)

train_df = pd.read_csv("binary_classification_training_set.csv")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive


In [2]:
# read in validation file
validation_df = pd.read_csv("binary_classification_validation_set.csv")

In [3]:
print(train_df.shape)
print(train_df.columns)
print(validation_df.shape)
print(validation_df.columns)

print(len(train_df[train_df["decision"] == "ACCEPTED"]))
print(len(train_df[train_df["decision"] == "REJECTED"]))
print(len(validation_df[validation_df["decision"] == "ACCEPTED"]))
print(len(validation_df[validation_df["decision"] == "REJECTED"]))

(20000, 19)
Index(['Unnamed: 0', 'patent_number', 'decision', 'title', 'abstract',
       'claims', 'background', 'summary', 'description', 'cpc_label',
       'ipc_label', 'filing_date', 'examiner_id', 'output',
       'application_invention_type', 'examiner_art_unit',
       'small_entity_indicator', 'aia_first_to_file', 'foreign'],
      dtype='object')
(5000, 19)
Index(['Unnamed: 0', 'patent_number', 'decision', 'title', 'abstract',
       'claims', 'background', 'summary', 'description', 'cpc_label',
       'ipc_label', 'filing_date', 'examiner_id', 'output',
       'application_invention_type', 'examiner_art_unit',
       'small_entity_indicator', 'aia_first_to_file', 'foreign'],
      dtype='object')
10000
10000
2500
2500


In [4]:
# Mapping the field output; 1 = ACCEPTED; 0 = REJECTED
train_df['output'] = 1
train_df.loc[train_df['decision'] == "REJECTED", 'output'] = 0

validation_df['output'] = 1
validation_df.loc[validation_df['decision'] == "REJECTED", 'output'] = 0

In [5]:
# check Rejected versus Accepted based on output field
print(len(train_df[train_df["output"] == 1]))
print(len(train_df[train_df["output"] == 0]))
print(len(validation_df[validation_df["output"] == 1]))
print(len(validation_df[validation_df["output"] == 0]))

10000
10000
2500
2500


In [6]:
# spot check train_df
train_df.head(5)

,Unnamed: 0,patent_number,decision,title,abstract,claims,background,summary,description,cpc_label,ipc_label,filing_date,examiner_id,output,application_invention_type,examiner_art_unit,small_entity_indicator,aia_first_to_file,foreign
0,17944,15187583,ACCEPTED,"FULLY INTEGRATED, DISPOSABLE TISSUE VISUALIZAT...",The present invention relates to a fully integ...,"1-11. (canceled) 12. A sterilized, integrated,...",<SOH> BACKGROUND OF THE INVENTION <EOH>1. Fiel...,<SOH> SUMMARY OF THE INVENTION <EOH>Embodiment...,CROSS-REFERENCE TO RELATED APPLICATIONS This a...,A61B107,A61B107,20160620,86346.0,1,Utility,3779,SMALL,true,False
1,1686,14906225,REJECTED,Compositions and Methods Comprising a Lipolyti...,The present invention provides lipolytic enzym...,1. A lipolytic enzyme variant or an active fra...,<SOH> BACKGROUND OF THE INVENTION <EOH>Lipolyt...,<SOH> SUMMARY OF THE INVENTION <EOH>The presen...,CROSS REFERENCE TO RELATED APPLICATIONS This a...,C12N920,C12N920,20160119,68762.0,0,Utility,1656,UNDISCOUNTED,true,False
2,3522,15012119,REJECTED,IMAGE DISPLAY DEVICE,An image display device of the present disclos...,1. An image display device comprising: a displ...,<SOH> BACKGROUND <EOH>1. Technical Field The p...,<SOH> SUMMARY <EOH>An image display device of ...,BACKGROUND 1. Technical Field The present disc...,G02B270172,G02B2701,20160201,99575.0,0,Utility,2626,UNDISCOUNTED,true,True
3,6639,15054201,REJECTED,ABUSE-PROOFED DOSAGE FORM,The invention relates to a dosage form that is...,1. An abuse-proofed dosage form thermoformed b...,NaN,<SOH> BRIEF DESCRIPTION OF THE DRAWING <EOH>FI...,This application is a continuation of U.S. Ser...,A61K31135,A61K31135,20160226,66231.0,0,Utility,1615,UNDISCOUNTED,false,True
4,18595,15108464,ACCEPTED,Process for the Preparation of Solid Particula...,A novel process for preparing vinyl aromatic p...,1. A process for the preparation of a solid pa...,<SOH> BACKGROUND OF THE INVENTION <EOH>Expanda...,<SOH> SUMMARY OF THE INVENTION <EOH>The object...,FIELD OF THE INVENTION The present invention r...,C08J920,C08J920,20160627,97379.0,1,Utility,1765,UNDISCOUNTED,true,True


In [7]:
# spot check train_df
validation_df.head(5)

,Unnamed: 0,patent_number,decision,title,abstract,claims,background,summary,description,cpc_label,ipc_label,filing_date,examiner_id,output,application_invention_type,examiner_art_unit,small_entity_indicator,aia_first_to_file,foreign
0,149,15227186,ACCEPTED,IMAGING DEVICE AND FOCUSING CONTROL METHOD,The present invention provides an imaging devi...,1. An imaging device comprising: an imaging el...,<SOH> BACKGROUND OF THE INVENTION <EOH>1. Fiel...,<SOH> SUMMARY OF THE INVENTION <EOH>In all of ...,CROSS-REFERENCE TO RELATED APPLICATIONS This a...,H04N523212,H04N5232,20160803,70534.0,1,Utility,2662.0,UNDISCOUNTED,true,True
1,4003,15129780,ACCEPTED,FOIL REMOVAL DEVICE AND A METHOD FOR REMOVING ...,Provided is a foil removal device and a method...,1-26. (canceled) 27. A foil removal device for...,<SOH> BACKGROUND <EOH>The invention relates to...,<SOH> SUMMARY OF THE INVENTION <EOH>According ...,BACKGROUND The invention relates to a foil rem...,B65H19286,B65H1928,20160927,61755.0,1,Utility,1745.0,UNDISCOUNTED,true,True
2,811,15234229,REJECTED,FUNGICIDAL COMPOSITION COMPRISING A PYRIDYLETH...,A composition comprising at least a pyridyleth...,1.-20. (canceled) 21. A composition comprising...,<SOH> BACKGROUND OF THE INVENTION <EOH>Interna...,<SOH> SUMMARY OF THE INVENTION <EOH>Accordingl...,CROSS-REFERENCE TO RELATED APPLICATION(S) The ...,A01N4340,A01N4340,20160811,62399.0,0,Utility,1627.0,UNDISCOUNTED,false,True
3,624,15233691,REJECTED,SEMICONDUCTOR MEMORY DEVICE,A semiconductor memory device includes a plura...,1. A semiconductor memory device comprising: a...,<SOH> BACKGROUND <EOH>A NAND type flash memory...,NaN,CROSS-REFERENCE TO RELATED APPLICATION This ap...,G11C160483,G11C1604,20160810,62558.0,0,Utility,2824.0,UNDISCOUNTED,true,True
4,3125,15125628,ACCEPTED,METHODS AND SYSTEMS FOR AUTOMATIC CREATION OF ...,Disclosed herein are methods and systems for a...,1. A method comprising: a first mobile radio b...,<SOH> BACKGROUND OF THE INVENTION <EOH>Million...,<SOH> BRIEF DESCRIPTION OF THE SEVERAL VIEWS O...,BACKGROUND OF THE INVENTION Millions of people...,H04W408,H04W408,20160913,62652.0,1,Utility,2648.0,UNDISCOUNTED,true,False


In [8]:
from transformers import AutoTokenizer

# PARAMETERS TO CHANGE HERE
COLUMN = "claims"  # Change here to encode different columns of the dataset
max_len = 512  
MODEL = 'bert-base-uncased' # Change here to choose a different base model

# Get the neccessary column values
train_column_encoded = train_df[COLUMN].values  
validation_column_encoded = validation_df[COLUMN].values

input_ids_train = []       # Store the encoded input values
attention_masks_train = [] # Store the encoded attention mask values

input_ids_validation = []       # Store the encoded input values
attention_masks_validation = [] # Store the encoded attention mask values

# tokenizer function for train set
def tokenize_patent_column_train(tokenizer):
  for sentence in train_column_encoded:
    encoded_dict = tokenizer.encode_plus(
                            sentence,              
                            max_length = max_len,.
                            truncation = True,
                            padding = 'max_length',
                            return_attention_mask = True,   
                            return_tensors = 'pt',     
                            ) 

    # Add the encoded sentence to the list.    
    input_ids_train.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks_train.append(encoded_dict['attention_mask'])

# tokenizer function for validation set
def tokenize_patent_column_validation(tokenizer):
  for sentence in validation_column_encoded:
    encoded_dict = tokenizer.encode_plus(
                            sentence,              
                            max_length = max_len,  
                            truncation = True,
                            padding = 'max_length',
                            return_attention_mask = True,   
                            return_tensors = 'pt',     
                            ) 

    input_ids_validation.append(encoded_dict['input_ids'])
    
    attention_masks_validation.append(encoded_dict['attention_mask'])


# Select which model being contemplated
if (MODEL == 'bert-base-uncased'):
  tokenizer = AutoTokenizer.from_pretrained(MODEL, do_lower_case=True)
  tokenize_patent_column_train(tokenizer)
  tokenize_patent_column_validation(tokenizer)  
elif (MODEL == 'distilbert-base-uncased'):
  tokenizer = AutoTokenizer.from_pretrained(MODEL, do_lower_case=True)
  tokenize_patent_column_train(tokenizer)
  tokenize_patent_column_validation(tokenizer)
elif (MODEL == 'roberta-base'):
  tokenizer = AutoTokenizer.from_pretrained(MODEL, do_lower_case=True)
  tokenize_patent_column_train(tokenizer)
  tokenize_patent_column_validation(tokenizer)
elif (MODEL == 'allenai/longformer-base-4096'):
  tokenizer = AutoTokenizer.from_pretrained(MODEL, do_lower_case=True)
  tokenize_patent_column_train(tokenizer)
  tokenize_patent_column_validation(tokenizer)
elif (MODEL == 'allenai/scibert_scivocab_uncased'):
  tokenizer = AutoTokenizer.from_pretrained(MODEL, do_lower_case=True)
  tokenize_patent_column_train(tokenizer)
  tokenize_patent_column_validation(tokenizer)
elif (MODEL == 'SciBERTClassifier'):
  tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased', do_lower_case=True)
  tokenize_patent_column_train(tokenizer)
  tokenize_patent_column_validation(tokenizer)

In [9]:
# Convert to tensor
labels_train = torch.tensor(train_df['output'].values)
input_ids_train = torch.stack(input_ids_train).squeeze()
attention_masks_train = torch.stack(attention_masks_train).squeeze()

labels_validation = torch.tensor(validation_df['output'].values)
input_ids_validation = torch.stack(input_ids_validation).squeeze()
attention_masks_validation = torch.stack(attention_masks_validation).squeeze()

In [10]:
# Implement the dataloader here

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# PARAMETERS TO CHANGE HERE
batch_size = 16
g = torch.Generator()
g.manual_seed(42)

h = torch.Generator()
h.manual_seed(42)


# Create the DataLoader for our training set.
train_data = TensorDataset(input_ids_train, attention_masks_train, labels_train)
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle = True, generator = g)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(input_ids_validation, attention_masks_validation, labels_validation)
validation_dataloader = DataLoader(validation_data, batch_size=batch_size, shuffle = True, generator = h)

In [11]:
# Implement the vanilla model here

from transformers.models.cvt.modeling_cvt import CrossEntropyLoss
from transformers import LongformerForSequenceClassification, RobertaForSequenceClassification, DistilBertForSequenceClassification, BertForSequenceClassification, AdamW, BertConfig, AutoModel
import torch.nn.functional as F
import torch.nn as nn

model = None

from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup

# SciBert defined here
class SciBERTClassifier(torch.nn.Module):
    def __init__(self, num_labels):
        super(SciBERTClassifier, self).__init__()
        
        self.SciBERT = BertModel.from_pretrained('allenai/scibert_scivocab_uncased')
        self.dropout_layer = nn.Dropout(0.25)
        self.classification_layer = torch.nn.Linear(768, num_labels)        
        
    def forward(self, input_ids, attention_mask, labels = None):
        if labels == None:        
          x = self.SciBERT(input_ids=input_ids)[1]
          x = self.dropout_layer(x)          
          logits = self.classification_layer(x)                    
          return [logits]
        else:
          x = self.SciBERT(input_ids=input_ids)[1]          
          x = self.dropout_layer(x)          
          logits = self.classification_layer(x)                              
          
          loss_func = CrossEntropyLoss()
          loss = loss_func(logits, labels)
          return [loss]          


if (MODEL == 'bert-base-uncased'):
  model = BertForSequenceClassification.from_pretrained(
      'bert-base-uncased', 
      num_labels = 2, 
                       
      output_attentions = False, 
      output_hidden_states = False, 
  )

elif (MODEL == 'distilbert-base-uncased'):
  model = DistilBertForSequenceClassification.from_pretrained(
      'distilbert-base-uncased', 
      num_labels = 2, 
      output_attentions = False, 
      output_hidden_states = False, 
  )
elif (MODEL == 'roberta-base'):
  model = RobertaForSequenceClassification.from_pretrained(
      'roberta-base', 
      num_labels = 2, 
      output_attentions = False,
      output_hidden_states = False, 
  )
elif (MODEL == 'allenai/longformer-base-4096'):
  model = LongformerForSequenceClassification.from_pretrained(
      'allenai/longformer-base-4096', 
      num_labels = 2, 
      output_attentions = False, 
      output_hidden_states = False, 
  )
elif (MODEL == 'SciBERTClassifier'):
  model = SciBERTClassifier(num_labels = 2)

model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [12]:
from transformers import get_linear_schedule_with_warmup

# PARAMETERS TO CHANGE HERE
epochs = 10

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8 
                )

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:
# Accuracy and time elapsed calculation
import numpy as np
import time
import datetime

# Implement F1 Score
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()        

    labels_flat = labels.flatten()
        
    # Change this part to calculate accuracy
    accuracy = np.sum(pred_flat == labels_flat) / len(labels_flat)
    
    # accuracy: 0
    # pred_flat: 1
    # labels_flat: 2
    return_package = [accuracy, pred_flat, labels_flat]
    
    return return_package


def format_time(elapsed):
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [14]:
# Training the model
!pip install torchmetrics

import random

from torchmetrics.classification import BinaryF1Score, BinaryPrecision, BinaryRecall

# Set device to GPU:
device = None
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))


loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    target_list = []
    output_list = []

    # Initial Validation


    if(epoch_i == 0):
      print("")
      print("Running Initial Validation...")

      t0 = time.time()

      model.eval()

      # Init variables for tracking 
      eval_loss, eval_accuracy = 0, 0
      nb_eval_steps, nb_eval_examples = 0, 0

      # Evaluate data for one epoch
      for batch in validation_dataloader:
          
          # Add batch to GPU
          batch = tuple(t.to(device) for t in batch)
          
          # Unpack the inputs from our dataloader
          b_input_ids, b_input_mask, b_labels = batch
          
          with torch.no_grad():        
              
              outputs = model(b_input_ids, 
                              attention_mask=b_input_mask)
          
          logits = outputs[0]

          # Move logits and labels to CPU
          logits = logits.detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()
          
          # Calculate the accuracy for this batch of test sentences.
          return_values = flat_accuracy(logits, label_ids)   
          
          eval_accuracy += return_values[0]
          output_list.append(return_values[1]) 
          target_list.append(return_values[2])   
        
          nb_eval_steps += 1   

      outputs_concat = torch.tensor(np.concatenate(output_list))  
      targets_concat = torch.tensor(np.concatenate(target_list))      
        
      metric = BinaryF1Score()
      print(">>>>>>>>>>> F1 Score: ", metric(outputs_concat, targets_concat))

      metric1 = BinaryPrecision()
      print(">>>>>>>>>>> Precision Score:", metric1(outputs_concat, targets_concat))

      metric2 = BinaryRecall()
      print(">>>>>>>>>>> Recall Score: ", metric2(outputs_concat, targets_concat))
                
      # Report the final accuracy for this validation run.
      print("  Accuracy: {0:.5f}".format(eval_accuracy/nb_eval_steps))
      print("  Validation took: {:}".format(format_time(time.time() - t0)))

      # Reset target output list to none again
      target_list = []
      output_list = []
      outputs_concat = None
      targets_concat = None
    
    # Training

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader.    
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
      
        model.zero_grad()        

        outputs = model(b_input_ids, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
               
        loss = outputs[0]
        
        total_loss += loss.item()

        loss.backward()

        # Clip the norm of the gradients to 1.0.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.5f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
        
    # Validation

    print("")
    print("Running Initial Validation...")

    t0 = time.time()

    model.eval()

    # Init variables for tracking 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():        
        
            outputs = model(b_input_ids, 
                            attention_mask=b_input_mask)
        
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Accuracy calculation
        return_values = flat_accuracy(logits, label_ids)                          
        eval_accuracy += return_values[0]
        output_list.append(return_values[1]) 
        target_list.append(return_values[2])   
      
        nb_eval_steps += 1
    
    outputs_concat = torch.tensor(np.concatenate(output_list))  
    targets_concat = torch.tensor(np.concatenate(target_list))
      
    metric = BinaryF1Score()
    print(">>>>>>>>>>> F1 Score: ", metric(outputs_concat, targets_concat))

    metric1 = BinaryPrecision()
    print(">>>>>>>>>>> Precision Score:", metric1(outputs_concat, targets_concat))

    metric2 = BinaryRecall()
    print(">>>>>>>>>>> Recall Score: ", metric2(outputs_concat, targets_concat))
              
    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.5f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

    # Reset target output list to none again
    target_list = []
    output_list = []
   
print("")
print("Training complete!")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
There are 1 GPU(s) available.
We will use the GPU: A100-SXM4-40GB

Running Initial Validation...
>>>>>>>>>>> F1 Score:  tensor(0.6666)
>>>>>>>>>>> Precision Score: tensor(0.5000)
>>>>>>>>>>> Recall Score:  tensor(0.9996)
  Accuracy: 0.50000
  Validation took: 0:00:37

======== Epoch 1 / 10 ========
Training...
  Batch    40  of  1,250.    Elapsed: 0:00:15.
  Batch    80  of  1,250.    Elapsed: 0:00:30.
  Batch   120  of  1,250.    Elapsed: 0:00:45.
  Batch   160  of  1,250.    Elapsed: 0:01:00.
  Batch   200  of  1,250.    Elapsed: 0:01:15.
  Batch   240  of  1,250.    Elapsed: 0:01:29.
  Batch   280  of  1,250.    Elapsed: 0:01:44.
  Batch   320  of  1,250.    Elapsed: 0:01:59.
  Batch   360  of  1,250.    Elapsed: 0:02:14.
  Batch   400  of  1,250.    Elapsed: 0:02:29.
  Batch   440  of  1,250.    Elapsed: 0:02:44.
  Batch   480  of  1,250.    Elapsed: 0:02:59.
  Batch   520  of  1,250.